## Transfer Raw Scene Viewing Encoding Files to USB (.ps1)

This script will treat session folders created within 10 hours as new data and copy it over to the USB. A prompt will appear asking for confirmation of the files that you would like to collect. It will then copy each session folder from each FinalRD folder over to their respective EncodingBlock folder on the USB. If an expected block is missing, it will throw an error before asking to proceed. Make sure to track this when it occurs. 

VERSION: `Scene_Encode_Eyetrack_Auto_Pull`

In [ ]:
$sourceBase = "C:\Users\at-risk\Documents\atrisk_nate\ARD2_Programming & Deployed_vF\Deployed Final"
$destinationBase = "E:\SceneViewing_DAY3"
$currentDate = Get-Date
$blocks = 1..4
$missingBlocks = @()
$dirInfo = @()


foreach ($block in $blocks) {
    $sourcePath = Join-Path -Path $sourceBase -ChildPath ("FinalR" + $block + "D\results")
    $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

    if ($rawdata.Count -eq 0) {
        $missingBlocks += $block
        continue
    } else {
        foreach ($dir in $rawdata) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
            $dirSizeMBStr = $dirSizeMB.ToString() + "M"
            $dirInfo += "[$($dir.Name): $($dirSizeMBStr)]"
        }
    }
}


$dirInfoStr = $dirInfo -join ", "
$userInput = Read-Host "Do you want to move the following directories to their respective folders in D:\SceneViewing_DAY3? Directories: $dirInfoStr [Y/N]"


if ($userInput -eq "Y") {
    if ($missingBlocks.Count -gt 0) {
        $missingBlockStr = $missingBlocks -join ", "
        $continueAnyway = Read-Host "Unexpected event: block(s) [$missingBlockStr] missing. Continue anyway? [Y/N]"
        if ($continueAnyway -ne "Y") {
            Write-Output "Operation aborted due to missing blocks."
            exit
        }
    }

    foreach ($block in $blocks) {
        if ($missingBlocks -contains $block) {
            Write-Output "Skipping block $block as it is missing."
            continue
        }

        $sourcePath = Join-Path -Path $sourceBase -ChildPath ("FinalR" + $block + "D\results")
        $destinationPath = Join-Path -Path $destinationBase -ChildPath ("Encoding_Block" + $block)
        $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }
        
        foreach ($dir in $rawdata) {
            $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to USB"
        }
    }
} else {
    Write-Output "Operation cancelled by user."
}
